# Load data

In [6]:
import scipy.io
import matplotlib.pyplot as plt

In [7]:

data = scipy.io.loadmat('data_for_cell77.mat')

In [25]:
box_size = data['boxSize'].flatten()
post = data['post'].flatten()
spiketrain = data['spiketrain'].flatten()
posx = data['posx'].flatten()
posx2 = data['posx2'].flatten()
posx_c = data['posx_c'].flatten()
posy = data['posy'].flatten()
posy2 = data['posy2'].flatten()
posy_c = data['posy_c'].flatten()
filt_eeg = data['filt_eeg'].flatten()
eeg_sample_rate = data['eeg_sample_rate'].flatten()
samplerate = data['sampleRate'].flatten()

# Model Fitting

In [30]:
import numpy as np
from scipy.signal import hilbert
from scipy.optimize import minimize
from sklearn.model_selection import KFold

In [32]:
def pos_map(pos, nbins, box_size):
    bins = np.linspace(box_size / (2 * nbins), box_size - box_size / (2 * nbins), nbins)

    pos_grid = np.zeros((len(pos), nbins**2))

    for idx, (x, y) in enumerate(pos):
        xcoor = np.argmin(np.abs(x - bins))
        ycoor = np.argmin(np.abs(y - bins))
        bin_idx = np.ravel_multi_index((nbins - ycoor - 1, xcoor), (nbins, nbins))
        pos_grid[idx, bin_idx] = 1

    return pos_grid, bins

def speed_map(posx, posy, nbins, sample_rate=50, max_speed=50):
    velx = np.diff(np.insert(posx, 0, posx[0]))  
    vely = np.diff(np.insert(posy, 0, posy[0]))
    speed = np.sqrt(velx**2 + vely**2) * sample_rate
    speed = np.clip(speed, None, max_speed)  

    speed_vec = np.linspace(max_speed / (2 * nbins), max_speed - max_speed / (2 * nbins), nbins)
    speed_grid = np.zeros((len(posx), len(speed_vec)))

    for i, s in enumerate(speed):
        idx = np.argmin(np.abs(s - speed_vec))
        speed_grid[i, idx] = 1

    return speed_grid, speed_vec, speed


def theta_map(filt_eeg, time_vec, sample_rate, nbins):
    hilb_eeg = hilbert(filt_eeg)
    phase = np.angle(hilb_eeg)  
    phase[phase < 0] += 2 * np.pi  

    phase_ind = np.round(time_vec * sample_rate).astype(int)
    phase_ind = phase_ind[phase_ind < len(filt_eeg)]  
    phase_time = phase[phase_ind]

    phase_vec = np.linspace(2 * np.pi / (2 * nbins), 2 * np.pi - 2 * np.pi / (2 * nbins), nbins)
    theta_grid = np.zeros((len(time_vec), nbins))

    for i, p in enumerate(phase_time):
        idx = np.argmin(np.abs(p - phase_vec))
        theta_grid[i, idx] = 1

    return theta_grid, phase_vec, phase_time

import numpy as np

def hd_map(posx, posx2, posy, posy2, nbins):
    direction = np.arctan2(posy2 - posy, posx2 - posx) + np.pi / 2
    direction[direction < 0] += 2 * np.pi  

    dirVec = np.linspace(2 * np.pi / (2 * nbins), 2 * np.pi - 2 * np.pi / (2 * nbins), nbins)

    hd_grid = np.zeros((len(posx), nbins))

    for i, d in enumerate(direction):
        idx = np.argmin(np.abs(d - dirVec))  
        hd_grid[i, idx] = 1  

    return hd_grid, dirVec, direction


In [26]:
n_pos_bins = 20
n_dir_bins = 18
n_speed_bins = 10
n_theta_bins = 18

In [31]:
from scipy.optimize import minimize
from sklearn.model_selection import KFold

def ln_model(theta, W):
    return np.exp(W @ theta)

def neg_log_likelihood(theta, W, spiketrain):
    r = ln_model(theta, W)
    return -np.sum(spiketrain * np.log(r) - r)  

def fit_model(W, spiketrain, num_folds=10):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    test_fit, train_fit = [], []
    
    for train_idx, test_idx in kf.split(W):
        W_train, W_test = W[train_idx], W[test_idx]
        spikes_train, spikes_test = spiketrain[train_idx], spiketrain[test_idx]

        theta_init = np.zeros(W.shape[1])
        res = minimize(neg_log_likelihood, theta_init, args=(W_train, spikes_train), method='BFGS')
        theta_opt = res.x

        train_fit.append(-neg_log_likelihood(theta_opt, W_train, spikes_train))
        test_fit.append(-neg_log_likelihood(theta_opt, W_test, spikes_test))

    return test_fit, train_fit, theta_opt


In [33]:
pos_grid, pos_bins = pos_map(np.column_stack((posx_c, posy_c)), n_pos_bins, box_size)
hd_grid, hd_bins, direction = hd_map(posx, posx2, posy, posy2, n_dir_bins)
speed_grid, speed_bins, speed = speed_map(posx_c, posy_c, n_speed_bins)
theta_grid, theta_bins, phase_time = theta_map(filt_eeg, post, eeg_sample_rate, n_theta_bins)

In [35]:
models = {
    "all": np.hstack([pos_grid, hd_grid, speed_grid, theta_grid]),
    "pos_hd_speed": np.hstack([pos_grid, hd_grid, speed_grid]),
    "pos_hd_theta": np.hstack([pos_grid, hd_grid, theta_grid]),
    "pos_speed_theta": np.hstack([pos_grid, speed_grid, theta_grid]),
    "hd_speed_theta": np.hstack([hd_grid, speed_grid, theta_grid]),
}

num_folds = 10
results = {}
# for name, features in models.items():
#     print(f"Fitting model: {name}")
#     results[name] = fit_model(features, spiketrain, num_folds)


In [ ]:
results['all'] = fit_model(models['all'], spiketrain, num_folds)

# Forward-Search Procedure To Find Best Model

# Tuning Curve Computation

# Plots